In [1]:
%load_ext autoreload 
%autoreload 2

In [39]:
from vehicle_detection_pipeline import *
import glob
import numpy as np
from sklearn.model_selection import train_test_split

In [78]:
car_img_list = glob.glob("../data/car/cars*/*")

In [79]:
notcar_img_list = glob.glob("../data/noncar/notcars*/*")

In [80]:
labels = np.hstack((np.ones(len(car_img_list)), 
                    np.zeros(len(notcar_img_list))))

In [81]:
labels.shape

(2321,)

## COLOR CLASSIFY

In [35]:
# read features
car_features = extract_featurres_from_img_list(car_img_list)
notcar_features = extract_featurres_from_img_list(notcar_img_list)

In [43]:
X = np.vstack((car_features, notcar_features)).astype(np.float64)

In [44]:
X.shape

(796, 864)

In [49]:
rand_state = np.random.randint(0, 100)
X_train, X_test, Y_train, Y_test = train_test_split(X, labels, test_size=0.2, random_state=rand_state)

In [50]:
X_scaler = get_scaler_from_feature_vectors(X_train)

In [51]:
scaled_X_train = X_scaler.transform(X_train)

In [52]:
scaled_X_test = X_scaler.transform(X_test)

In [53]:
from sklearn.svm import LinearSVC

In [55]:
svc = LinearSVC()
svc.fit(scaled_X_train, Y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [56]:
print (svc.score(scaled_X_test, Y_test))

0.9875


In [57]:
svc.predict(scaled_X_test[0:10])

array([ 1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.])

In [62]:
print (Y_test[50:100] - svc.predict(scaled_X_test[50:100]))

[ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


## HOG CLASSIFY

In [82]:
car_hog_features = extract_hog_features(car_img_list, cspace='RGB', cell_per_block=2, pix_per_cell=8, oritentation_bins=12, hog_channel=3)

/home/chuondao/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [83]:
noncar_hog_features = extract_hog_features(notcar_img_list, cspace='RGB', cell_per_block=2, pix_per_cell=8, oritentation_bins=12, hog_channel=3)

/home/chuondao/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [84]:
X_hog = np.vstack((car_hog_features, noncar_hog_features)).astype(np.float64)

In [85]:
rand_state = np.random.randint(0, 100)
X_train, X_test, Y_train, Y_test = train_test_split(X_hog, labels, test_size=0.2, random_state=rand_state)

In [86]:
X_scaler = get_scaler_from_feature_vectors(X_train)

In [87]:
scaled_X_train = X_scaler.transform(X_train)

In [88]:
scaled_X_test = X_scaler.transform(X_test)

In [89]:
svc_hog = LinearSVC()

In [90]:
svc_hog.fit(scaled_X_train, Y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [91]:
svc_hog.score(scaled_X_test, Y_test)

0.96989247311827953

In [93]:
svc_hog.predict(scaled_X_test[0:150]) - Y_test[0:150]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0., -1.,  0.,  0.,  0.,  0.,  0.])

## SLIDING WINDOWS